In [1]:
import sys
sys.path.append('..')

from glob import glob

from etl import use_existing_data


In [2]:
def get_tickers_from_dir(dir: str):
    return [match.split("/")[-1] for match in glob(f'../select/{dir}*/*')]

large = get_tickers_from_dir('large')        
mid = get_tickers_from_dir('mid')       

In [5]:
large_dfs = use_existing_data("../select/large_0_50", large, "")
mid_dfs = use_existing_data("../select/mid_0_50", mid, "")